## Initialization

There are several ways to draw background maps with Python. For a complete review, visit the [map section](https://www.python-graph-gallery.com/map) of the gallery

This example uses the `Basemap` library. Let's initialize a map of the world as explained in [this post](https://www.python-graph-gallery.com/281-basic-map-with-basemap).

In [1]:
# libraries
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from mpl_toolkits.basemap import Basemap
from thefuzz import fuzz

# Set the plot size for this notebook:
plt.rcParams["figure.figsize"]=18,10

C:\ProgramData\Miniconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# alliance = pd.read_csv("./version4.1_csv/alliance_v4.1_by_dyad_yearly.csv")
alliance = pd.read_csv("./version4.1_csv/alliance_v4.1_by_dyad.csv")

## Helper functions

In [3]:
def is_float(x: str):
    """
    Determine if the string `x` can be converted to float
    """
    try:
        float(x)
        return True
    except ValueError:
        return False

def has_char(x: str):
    """
    Check if string `x` contains any alphabetical characters
    """
    m = re.search('[a-zA-Z]', x)
    return m is not None

def str_best_match(s: str, strings):
    """
    Find the best match of string `s` in the list of strings.
    """
    assert isinstance(s, str) and isinstance(strings, list)
    s = s.lower()
    strings = [ss.lower() for ss in strings]
    candidates = [ss for ss in strings if ss.find(s) >= 0 or s.find(ss) >= 0]
    if len(candidates) == 0:
        return None
    match_score = [(fuzz.ratio(s, ss), ss) for ss in candidates]
    best_match = sorted(match_score, key=lambda x: x[0])[-1]
    idx = strings.index(best_match[1])
    return idx

## 🔗  Great Circle

> A great circle is the intersection of the sphere and a plane that passes through the center point of the sphere. [Wikipedia](https://en.wikipedia.org/wiki/Great_circle)

Basically, a great circle shows the shortest path between 2 locations, knowing that our planet is a sphere. This path is not a straight line but an arc, which gives a much better appearance to the map.

Let's add a connection between London and New York. This is quite straightforward with `Basemap` thanks to the `drawgreatcircle()` function.

## Alliance Map Class

Create a map class that is a subclass of `Basemap`. Add some helper functions.

In [4]:

class MyBasemap(Basemap):
    def __init__(self, *args, **kwargs):
        super().__init__()
        # load country attributes such as longitude, latitude, and BGN names
        self.countries_data = pd.read_csv("./version4.1_csv/cow.txt", sep=";", skiprows=28)
        self.countries_data.BGN_proper = self.countries_data.BGN_proper.apply(lambda x: x.strip(" "))
        self.countries_data.BGN_name = self.countries_data.BGN_name.apply(lambda x: x.strip(" "))
        self.BGN_name = self.countries_data.BGN_name.tolist()
        self.BGN_proper = self.countries_data.BGN_proper.tolist()
        self.countries_name = self.BGN_name + self.BGN_proper
        self.decide_countries_no_map()
        self.labelled_countries = set()

    def decide_countries_no_map(self, d: int=1, min_size: float=1e4):
        """
        Decide which countries can be labelled on the map. 
        Only label those with a land area larger than `min_size`.
        :param d: the distance to the edge of the map, if a country is too close to the edge, not to label it 
        """
        self.countries_on_map = set()
        data = self.countries_data
        data = data[
            (data.latitude > self.llcrnrlat+d) & 
            (data.latitude < self.urcrnrlat-d) & 
            (data.longitude > self.llcrnrlon+d) & 
            (data.longitude < self.urcrnrlon-d)
        ]
        for ix, country in data.iterrows():
            if has_char(country.BGN_proper.lower()) and is_float(country.land_total) and float(country.land_total) >= min_size:
                self.countries_on_map.add(country.BGN_proper.lower())
                self.countries_on_map.add(country.BGN_name.lower())
                # print(country.BGN_proper.lower(), pos)
        self.countries_on_map = list(self.countries_on_map)

    def best_match_country(self, country: str):
        """
        Find the best match of `country` in self.countries_name
        """
        match_idx = str_best_match(country.lower(), self.countries_name)
        if match_idx is None:
            return
        name = self.countries_name[match_idx]
        if match_idx >= len(self.BGN_name):
            match_idx -= len(self.BGN_name)
        return match_idx, name

    def print_countries(self, country: str, max_len: int=30, fontsize: int=10):
        """
        Label `country` with their name on the map. Only print 
        the first `max_len` char of the country names.
        """
        res = self.best_match_country(country)
        if res is None:
            return
        match_idx, name = res
        country_row = self.countries_data.iloc[match_idx]
        if country_row.BGN_name in self.labelled_countries:
            return
        self.labelled_countries.add(country_row.BGN_name)
        pos = self(country_row.longitude, country_row.latitude)
        pos_text = self(country_row.longitude, country_row.latitude + 2)
        # plt.text(*pos, s=country[:max_len], fontsize=fontsize)
        plt.annotate(
            country[:max_len], 
            xy=pos_text, 
            verticalalignment='center', 
            horizontalalignment='center', 
            fontsize=fontsize
        )
        plt.plot([pos[0]], [pos[1]], marker="o", markersize=3, markeredgecolor="blue", markerfacecolor="blue")

    def get_country_row(self, country):
        """
        Query the attributes of `country`
        """
        ct = country.lower()
        match_i = str_best_match(ct, self.countries_on_map)
        if match_i is None:
            return None
        match_idx, name = self.best_match_country(ct)
        if name.lower() != self.countries_on_map[match_i]:
            return
        row = self.countries_data.iloc[match_idx]
        # print(ct, row)
        return row

    def connect_countries(self, c0: str, c1: str):
        """
        Draw a great circle between the two countries `c0` and `c1` if 
        they are labelled on the map and their positions (latitude, longitude) can be found.
        """
        r0 = self.get_country_row(c0)
        r1 = self.get_country_row(c1)
        if r0 is None or r1 is None:
            return
        self.print_countries(c0)
        self.print_countries(c1)
        coord0 = self(r0.longitude.item(), r0.latitude.item())
        coord1 = self(r1.longitude.item(), r1.latitude.item())
        coord = coord0 + coord1
        # print(r0.BGN_name, r1.BGN_name, coord)
        if (abs(coord[0]) + abs(coord[2])) > 170 and abs(coord[0] + coord[2]) < 80:  # abs(coord[0] - coord[2]) > 180 and
            # the great circle in this case is drawn across the edges of the map,
            # so we use straight line to connect
            plt.plot(coord[0::2], coord[1::2], '-', linewidth=1, color='#69b3a2')
        else:
            self.drawgreatcircle(*coord, linewidth=1, color='#69b3a2')

In [5]:
def get_year_map(m, year, key="dyad_st_year"):
    """
    Draw a great circle on the Basemap `m` between the 
    countries that established alliance in `year`.
    """
    year_data = alliance.loc[alliance[key] == year]
    for it, (idx, row) in enumerate(year_data.iterrows()):
        # print(row.state_name1, row.state_name2)
        m.connect_countries(row.state_name1, row.state_name2)

In [6]:
def plot_alliance(year, show_fig=True, save_fig=False):
    # Background map
    plt.figure()
    m=MyBasemap(llcrnrlon=-179, llcrnrlat=-60, urcrnrlon=179, urcrnrlat=70,  projection='merc')
    m.drawmapboundary(fill_color='white', linewidth=0)
    m.drawcoastlines(linewidth=0.1)
    m.drawcountries(linewidth=0.2)
    m.fillcontinents(color='#f2f2f2', alpha=0.7)
    m.drawcoastlines(linewidth=0.1, color="white")

#     m.connect_countries("United States", "Canada")
#     m.connect_countries('United States', 'Dominican Republic')
#     m.connect_countries('Canada', 'Japan')
#     m.connect_countries('Canada', 'North Korea')
    get_year_map(m, year)
    plt.title(f"New alliances established in {year}")
    plt.tight_layout()
    if save_fig:
        plt.savefig(f"./img/{year}.png")
    if not show_fig:
        plt.close('all')

In [7]:
for year in range(1816, 2013):
    plot_alliance(year, False, True)
# plot_alliance(1999)